# Classification - Comparison of KNN, Logistic Regression, Decision Tree & SVM

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
# from IPython.display import Image 
# from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
# from pandas.io.json import json_normalize

# import folium # plotting library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
venues = pd.read_excel('venues.xlsx', index_col=0)
venues.shape

(12889, 10)

In [3]:
for index, row in venues.iterrows():
    if row['m_id'] == row["Venue ID"] or row['s_id'] == row["Venue ID"]:
        venues.drop([index], inplace=True)
venues.shape

(12699, 10)

create one more column to identify spots to group data

In [4]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Type'] = venues['Type'] 
onehot['Latitude'] = venues['Latitude']
onehot['Longitude'] = venues['Longitude']

In [5]:
onehot['dummyID'] = onehot['Latitude'].astype(str) + onehot['Longitude'].astype(str)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [6]:
grouped = onehot.groupby('dummyID').mean().reset_index()


In [7]:
grouped.head()

,dummyID,ATM,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Belgian Restaurant,Betting Shop,Bistro,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Bus Station,Bus Stop,Cable Car,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donburi Restaurant,Dongbei Restaurant,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Exhibit,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fujian Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,Office,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Port,Portuguese Restaurant,Pub,Public Art,Racecourse,Ramen Restaurant,Record Shop,Recreation Center,Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Social Club,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,T

Assign feature set X and labels Y

In [8]:
data = grouped
Y = data['Type']
X = data.drop(columns=['Type', 'Latitude', 'Longitude','dummyID'])

In [9]:
X.head()

,ATM,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Belgian Restaurant,Betting Shop,Bistro,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Bus Station,Bus Stop,Cable Car,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donburi Restaurant,Dongbei Restaurant,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Exhibit,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fujian Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,Office,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Port,Portuguese Restaurant,Pub,Public Art,Racecourse,Ramen Restaurant,Record Shop,Recreation Center,Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Social Club,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Takoyaki 

### Train Test Split 
We use K-Fold for cross-validation


In [10]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.2)

kf = KFold(n_splits=4, shuffle=True)
folds = kf.get_n_splits(X)


<div id="classification">
    <h2>Classification</h2>
</div>

<h3>K nearest neighbor (KNN)</h3>

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

#### What about other K?
K in KNN, is the number of nearest neighbors to examine. It is supposed to be specified by the User. So, how can we choose right value for K?
The general solution is to reserve a part of your data for testing the accuracy of the model. Then chose k =1, use the training part for modeling, and calculate the accuracy of prediction using all samples in your test set. Repeat this process, increasing the k, and see which k is the best for your model.

We can calculate the accuracy of KNN for different Ks.

In [12]:
Ks = 10
mean_acc = np.zeros((Ks-1))
for n in range(1,Ks):

    mean_acc_kf = [0] * folds
    
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        X_train_kf, X_test_kf = X.iloc[train_index], X.iloc[test_index]
        y_train_kf, y_test_kf = Y.iloc[train_index], Y.iloc[test_index]    
    
        #Train Model and Predict  
        neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train_kf,y_train_kf)
        yhat=neigh.predict(X_test_kf)
        
        mean_acc_kf[i] = metrics.accuracy_score(y_test_kf, yhat)

        
    mean_acc[n-1] = sum(mean_acc_kf) / len(mean_acc_kf)



In [13]:
knn_acc = pd.DataFrame(columns = ['K', 'Accuracy Score'])
knn_acc['K'] = range(1,Ks)
knn_acc['Accuracy Score'] = mean_acc
knn_acc

,K,Accuracy Score
0,1,0.555849
1,2,0.545188
2,3,0.524354
3,4,0.559126
4,5,0.524208
5,6,0.534918
6,7,0.548856
7,8,0.573552
8,9,0.524795


<hr>

<div id="modeling">
    <h2>Decisiuon Tree - Modeling</h2>
</div>

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
max_depth = 20
mean_acc = np.zeros((max_depth-1))

for n in range(2,max_depth+1):
    
    mean_acc_kf = [0] * folds
    
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        X_train_kf, X_test_kf = X.iloc[train_index], X.iloc[test_index]
        y_train_kf, y_test_kf = Y.iloc[train_index], Y.iloc[test_index]    
    
        #Train Model and Predict  
        tree = DecisionTreeClassifier(criterion="entropy", max_depth=n).fit(X_train_kf,y_train_kf)
        yhat=tree.predict(X_test_kf)
        
        mean_acc_kf[i] = metrics.accuracy_score(y_test_kf, yhat)

        
    mean_acc[n-2] = sum(mean_acc_kf) / len(mean_acc_kf)

In [16]:
tree_acc = pd.DataFrame(columns = ['Max Depth', 'Accuracy Score'])
tree_acc['Max Depth'] = range(2,max_depth+1)
tree_acc['Accuracy Score'] = mean_acc
tree_acc

,Max Depth,Accuracy Score
0,2,0.594630
1,3,0.650186
2,4,0.615072
3,5,0.555996
4,6,0.576976
5,7,0.538194
6,8,0.538341
7,9,0.555996
8,10,0.552817
9,11,0.538683


<h2 id="modeling">Modeling (Logistic Regression)</h2>

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import log_loss

Cs = [0.01, 0.1, 1]
solvers = ['newton-cg', 'lbfgs', 'sag', 'saga']

acc_table = pd.DataFrame()

for x in solvers:
    for y in Cs:

        accuracy = []
        for i, (train_index, test_index) in enumerate(kf.split(X)):
            X_train_kf, X_test_kf = X.iloc[train_index], X.iloc[test_index]
            y_train_kf, y_test_kf = Y.iloc[train_index], Y.iloc[test_index]    

            #Train Model and Predict  
            LR = LogisticRegression(C=y, solver=x, multi_class='multinomial').fit(X_train_kf,y_train_kf)
            yhat = LR.predict(X_test_kf)
            yhat_prob = LR.predict_proba(X_test_kf)

            accuracy.append(metrics.accuracy_score(y_test_kf, yhat))
        
        mean_a = sum(accuracy) / len(accuracy)
        
        row = pd.DataFrame([[x, y, mean_a]], columns = ['Solver', 'C', 'Accuracy Score'])       
        acc_table = acc_table.append(row,ignore_index=True)


In [18]:
acc_table

,Solver,C,Accuracy Score
0,newton-cg,0.01,0.464984
1,newton-cg,0.10,0.472124
2,newton-cg,1.00,0.569885
3,lbfgs,0.01,0.464935
4,lbfgs,0.10,0.465033
5,lbfgs,1.00,0.563380
6,sag,0.01,0.464887
7,sag,0.10,0.472222
8,sag,1.00,0.556289
9,saga,0.01,0.465180


<h2> SVM with Scikit-learn</h2>

The SVM algorithm offers a choice of kernel functions for performing its processing. Basically, mapping data into a higher dimensional space is called kernelling. The mathematical function used for the transformation is known as the kernel function, and can be of different types, such as:

    1.Linear
    2.Polynomial
    3.Radial basis function (RBF)
    4.Sigmoid
Each of these functions has its characteristics, its pros and cons, and its equation, but as there's no easy way of knowing which function performs best with any given dataset, we usually choose different functions in turn and compare the results. Let's just use the default, RBF (Radial Basis Function) for this lab.

In [19]:
from sklearn import svm
from sklearn.metrics import f1_score

kernels = ['linear', 'poly', 'rbf', 'sigmoid',]
As = np.zeros(4)
a=[]

for i, k in enumerate(kernels):
    
    for j, (train_index, test_index) in enumerate(kf.split(X)):
        X_train_kf, X_test_kf = X.iloc[train_index], X.iloc[test_index]
        y_train_kf, y_test_kf = Y.iloc[train_index], Y.iloc[test_index]       
        cls = svm.SVC(kernel=k).fit(X_train_kf, y_train_kf) 
        yhat = cls.predict(X_test_kf)    
    
        a.append(metrics.accuracy_score(y_test_kf, yhat))


    As[i] = sum(a) / len(a)


/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/jupyterlab/conda/lib/python3.6/site-packages/skle

In [20]:
acc_df = pd.DataFrame()
acc_df['Kernel'] = kernels
acc_df['Accuracy Score'] = As
acc_df

,Kernel,Accuracy Score
0,linear,0.506993
1,poly,0.485842
2,rbf,0.478922
3,sigmoid,0.475389


### Further examine for Logistic regression & KNN

In [39]:
LR = LogisticRegression(C=1, solver='saga', multi_class='multinomial').fit(X_train_kf,y_train_kf)
yhat = LR.predict(X)
yhat

array([2, 0, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2,
       0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0,
       2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2])

In [40]:
neigh = KNeighborsClassifier(n_neighbors = 8).fit(X_train_kf,y_train_kf)
yhat = neigh.predict(X)
yhat

array([2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 0, 0, 2, 1, 0,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1,
       2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 0, 1, 1, 1, 2,
       1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2,
       0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 2])